# MNIST 손글씨 - CNN v1
- 2 Conv2D layers
- 1 Fully Connected Network layer
- 1 Output layer

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

* 데이터 전처리

In [2]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# X data : 0~1 사이의 값, 3차원 --> 4차원   # Conv2D의 경우 3차원의 데이터를 받는다(즉, X데이터는 4차원이어야함)(60000, 28, 28, 1)
X_train = X_train.reshape(-1, 28, 28, 1) / 255
X_test = X_test.reshape(-1, 28, 28, 1) / 255

In [4]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

* 모델 정의, 설정, 학습, 평가

In [5]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
model = Sequential([
                    Conv2D(32, kernel_size=(3, 3), input_shape = (28, 28, 1), activation = 'relu'),
                    Conv2D(64, (3, 3), activation = 'relu'), # default로 filter 다음에 kernel_size가 오므로 저렇게 써도 된다
                    Flatten(),
                    Dense(512, activation = 'relu'),
                    Dense(10, activation = 'softmax')
])
model.summary()

# pedding이 없으므로 28*28이 26*26으로 줄고, 24*24로 줄은 것을 확인할 수 있다

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 512)               18874880  
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 18,898,826
Trainable params: 18,898,826
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [8]:
model_path = 'best-mnist-cnn-v1.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=10)

In [9]:
hist = model.fit(
    X_train, Y_train, validation_split = 0.2, epochs = 100, batch_size = 200,
    callbacks = [checkpoint, early_stop], verbose = 0
)

Epoch 1/100
240/240 [==============================] - ETA: 0s - loss: 0.1520 - accuracy: 0.9534
Epoch 00001: val_loss improved from inf to 0.05703, saving model to best-mnist-cnn-v1.h5
240/240 [==============================] - 25s 51ms/step - loss: 0.1520 - accuracy: 0.9534 - val_loss: 0.0570 - val_accuracy: 0.9830
Epoch 2/100
239/240 [============================>.] - ETA: 0s - loss: 0.0353 - accuracy: 0.9893
Epoch 00002: val_loss improved from 0.05703 to 0.05333, saving model to best-mnist-cnn-v1.h5
240/240 [==============================] - 11s 47ms/step - loss: 0.0352 - accuracy: 0.9893 - val_loss: 0.0533 - val_accuracy: 0.9856
Epoch 3/100
240/240 [==============================] - ETA: 0s - loss: 0.0172 - accuracy: 0.9948
Epoch 00003: val_loss improved from 0.05333 to 0.04869, saving model to best-mnist-cnn-v1.h5
240/240 [==============================] - 12s 49ms/step - loss: 0.0172 - accuracy: 0.9948 - val_loss: 0.0487 - val_accuracy: 0.9870
Epoch 4/100
239/240 [==============

In [10]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.0431 - accuracy: 0.9876


[0.043144140392541885, 0.9876000285148621]